# How to work with config in CoolGraph <a name="toc"></a>
1. [Without changing files (runner.cfg)](#p2)
2. [Making dict with config parameters](#p3)
3. [Passing path to your config file](#p4)

# 1. Without changing files (runner.cfg) <a id="p2"></a>

#### Runner and HypeRunner have default config parameters but they can be changed by changing `runner.cfg` dict

In [2]:
from cool_graph.runners import Runner, HypeRunner
from torch_geometric import datasets
import torch

In [2]:
# taking Amazon Computers as an Example
dataset = datasets.Amazon(root='./data/Amazon', name='Computers')
data = dataset.data
data

Data(x=[13752, 767], edge_index=[2, 491722], y=[13752], index=[13752])

In [3]:
# Runner has attributes: 
# config: Union[omegaconf.dictconfig.DictConfig, NoneType] = None,
# config_path: Union[str, NoneType] = None,

In [4]:
# leaving those attributes empty
runner = Runner(data)

In [5]:
type(runner.cfg)

dict

In [6]:
runner.cfg.keys()

dict_keys(['model_params', 'training', 'logging', 'metrics'])

In [7]:
runner.cfg["model_params"]

{'conv_type': 'GraphConv',
 'activation': 'leakyrelu',
 'lin_prep_dropout_rate': 0.4,
 'lin_prep_len': 1,
 'lin_prep_size_common': 512,
 'lin_prep_sizes': [256],
 'lin_prep_weight_norm_flag': True,
 'graph_conv_weight_norm_flag': True,
 'n_hops': 2,
 'conv1_aggrs': {'mean': 128, 'max': 64, 'add': 32},
 'conv1_dropout_rate': 0.2,
 'conv2_aggrs': {'mean': 64, 'max': 32, 'add': 16},
 'conv2_dropout_rate': 0.2}

In [8]:
runner.cfg["training"]

{'device': 'cuda:0',
 'targets': ['y'],
 'num_neighbors': [25, 15],
 'batch_size': 250,
 'initial_lr': 0.0023,
 'weight_decay': 0.0001,
 'n_epochs': 21,
 'eval_freq': 5,
 'scheduler_type': 'MultiStepLR',
 'scheduler_params': {'milestones': [10, 20, 35, 50, 70, 90, 105],
  'gamma': 0.25},
 'loss': {'name': 'CrossEntropyLoss',
  'label_smoothing': False,
  'target_weights': {'y': 1},
  'group_weights': [1],
  'fill_value': -100}}

In [9]:
runner.cfg["metrics"]

['accuracy', 'cross_entropy', 'f1_weighted']

In [14]:
# The way to change config without changing .yaml files
# is to simply change runner.cfg dictionary
runner.cfg["training"]["num_neighbors"] = [-1, -1]

In [20]:
runner.cfg["training"]["n_epochs"] = 15

In [21]:
# checking the changes
runner.cfg["training"]

{'device': 'cuda:0',
 'targets': ['y'],
 'num_neighbors': [-1, -1],
 'batch_size': 250,
 'initial_lr': 0.0023,
 'weight_decay': 0.0001,
 'n_epochs': 15,
 'eval_freq': 5,
 'scheduler_type': 'MultiStepLR',
 'scheduler_params': {'milestones': [10, 20, 35, 50, 70, 90, 105],
  'gamma': 0.25},
 'loss': {'name': 'CrossEntropyLoss',
  'label_smoothing': False,
  'target_weights': {'y': 1},
  'group_weights': [1],
  'fill_value': -100}}

In [22]:
# success! We've changed config

In [23]:
res = runner.run()

2024-05-20 15:52:17.855 | INFO     | cool_graph.train.helpers:eval_epoch:199 - test:
 {'accuracy': 0.506, 'cross_entropy': 1.264, 'f1_weighted': 0.453, 'calc_time': 0.006, 'main_metric': 0.506}
2024-05-20 15:52:18.674 | INFO     | cool_graph.train.helpers:eval_epoch:199 - train:
 {'accuracy': 0.499, 'cross_entropy': 1.278, 'f1_weighted': 0.443, 'calc_time': 0.014, 'main_metric': 0.499}
2024-05-20 15:52:25.697 | INFO     | cool_graph.train.helpers:eval_epoch:199 - test:
 {'accuracy': 0.891, 'cross_entropy': 0.336, 'f1_weighted': 0.884, 'calc_time': 0.005, 'main_metric': 0.891}
2024-05-20 15:52:26.575 | INFO     | cool_graph.train.helpers:eval_epoch:199 - train:
 {'accuracy': 0.908, 'cross_entropy': 0.279, 'f1_weighted': 0.903, 'calc_time': 0.015, 'main_metric': 0.908}
2024-05-20 15:52:33.493 | INFO     | cool_graph.train.helpers:eval_epoch:199 - test:
 {'accuracy': 0.92, 'cross_entropy': 0.276, 'f1_weighted': 0.919, 'calc_time': 0.005, 'main_metric': 0.92}
2024-05-20 15:52:34.212 | INFO

In [24]:
# as we see, changes have been applied
res["best_loss"]

{'accuracy': 0.92,
 'cross_entropy': 0.276,
 'f1_weighted': 0.919,
 'calc_time': 0.005,
 'main_metric': 0.92,
 'epoch': 10}

# 2. Making dict with config parameters <a id="p3"></a>

#### Another way is to make python dictionary with your config, pass it to config creator `OmegaConf` and then pass resulting config into runner as an attribute

In [26]:
cfg_dict = {'model_params': {'conv_type': 'GraphConv',
  'activation': 'leakyrelu',
  'lin_prep_dropout_rate': 0.4,
  'lin_prep_len': 1,
  'lin_prep_size_common': 512,
  'lin_prep_sizes': [256],
  'lin_prep_weight_norm_flag': True,
  'graph_conv_weight_norm_flag': True,
  'n_hops': 2,
  'conv1_aggrs': {'mean': 128, 'max': 64, 'add': 32},
  'conv1_dropout_rate': 0.2,
  'conv2_aggrs': {'mean': 64, 'max': 32, 'add': 16},
  'conv2_dropout_rate': 0.2},
 'training': {'device': 'cuda:0',
  'targets': ['y0', 'y1', 'y2', 'y3', 'y4', 'y5', 'y6', 'y7', 'y8', 'y9'],
  'num_neighbors': [25, 15],
  'batch_size': 250,
  'initial_lr': 0.0023,
  'weight_decay': 0.0001,
  'n_epochs': 1,
  'eval_freq': 5,
  'scheduler_type': 'MultiStepLR',
  'scheduler_params': {'milestones': [10, 20, 35, 50, 70, 90, 105],
   'gamma': 0.25},
  'loss': {'name': 'CrossEntropyLoss',
   'label_smoothing': False,
   'target_weights': {'y': 1},
   'group_weights': [1],
   'fill_value': -100}},
 'logging': {'mlflow_experiment_name': 'coolgraph',
  'use_mlflow': False,
  'checkpoint_dir': './checkpoints'},
 'metrics': ['accuracy', 'cross_entropy', 'f1_weighted']}

In [27]:
# creating config from our dictionary for Runner using OmegaConf

In [28]:
from omegaconf import OmegaConf
cfg = OmegaConf.create(cfg_dict)

In [29]:
cfg

{'model_params': {'conv_type': 'GraphConv', 'activation': 'leakyrelu', 'lin_prep_dropout_rate': 0.4, 'lin_prep_len': 1, 'lin_prep_size_common': 512, 'lin_prep_sizes': [256], 'lin_prep_weight_norm_flag': True, 'graph_conv_weight_norm_flag': True, 'n_hops': 2, 'conv1_aggrs': {'mean': 128, 'max': 64, 'add': 32}, 'conv1_dropout_rate': 0.2, 'conv2_aggrs': {'mean': 64, 'max': 32, 'add': 16}, 'conv2_dropout_rate': 0.2}, 'training': {'device': 'cuda:0', 'targets': ['y0', 'y1', 'y2', 'y3', 'y4', 'y5', 'y6', 'y7', 'y8', 'y9'], 'num_neighbors': [25, 15], 'batch_size': 250, 'initial_lr': 0.0023, 'weight_decay': 0.0001, 'n_epochs': 1, 'eval_freq': 5, 'scheduler_type': 'MultiStepLR', 'scheduler_params': {'milestones': [10, 20, 35, 50, 70, 90, 105], 'gamma': 0.25}, 'loss': {'name': 'CrossEntropyLoss', 'label_smoothing': False, 'target_weights': {'y': 1}, 'group_weights': [1], 'fill_value': -100}}, 'logging': {'mlflow_experiment_name': 'coolgraph', 'use_mlflow': False, 'checkpoint_dir': './checkpoints

In [30]:
# now passing cfg as a parameter into runner

In [31]:
runner_hand_cfg = Runner(data, config=cfg)

In [32]:
# success
res_hand_cfg = runner_hand_cfg.run()

Sample data: 100%|██████████| 14/14 [00:02<00:00,  6.71it/s]
2024-05-20 15:53:38.187 | INFO     | cool_graph.train.helpers:eval_epoch:199 - test:
 {'accuracy': 0.641, 'cross_entropy': 0.996, 'f1_weighted': 0.586, 'calc_time': 0.004, 'main_metric': 0.641}
2024-05-20 15:53:38.881 | INFO     | cool_graph.train.helpers:eval_epoch:199 - train:
 {'accuracy': 0.622, 'cross_entropy': 1.017, 'f1_weighted': 0.562, 'calc_time': 0.012, 'main_metric': 0.622}


# 3. Passing path to your config file <a id="p4"></a>

In [ ]:
# if you are not working with the library in dev mode and as a result you do not have the configs folder
# can be copied using the terminal with command:
# get_config --configs <path/to/config/where/you/need/it>

In [33]:
!cat ../cool_graph/config/in_memory_data.yaml

# this config for runner in notebook (no attrs on edges)
defaults:
  - model_params: graphconv
  - training: in_memory_data
  - logging: in_memory_data
  - metrics: in_memory_data

In [34]:
# you can also add the path to the configs (by making any changes directly to the yaml files
# i.e. I only need to change the confing training: in_memory_data

In [35]:
runner_with_path = Runner(data, config_path="../cool_graph/config/in_memory_data.yaml")

In [36]:
# we see the same dictionary
runner_with_path.cfg

{'model_params': {'conv_type': 'GraphConv',
  'activation': 'leakyrelu',
  'lin_prep_dropout_rate': 0.4,
  'lin_prep_len': 1,
  'lin_prep_size_common': 512,
  'lin_prep_sizes': [256],
  'lin_prep_weight_norm_flag': True,
  'graph_conv_weight_norm_flag': True,
  'n_hops': 2,
  'conv1_aggrs': {'mean': 128, 'max': 64, 'add': 32},
  'conv1_dropout_rate': 0.2,
  'conv2_aggrs': {'mean': 64, 'max': 32, 'add': 16},
  'conv2_dropout_rate': 0.2},
 'training': {'device': 'cuda:0',
  'targets': ['y'],
  'num_neighbors': [25, 15],
  'batch_size': 250,
  'initial_lr': 0.0023,
  'weight_decay': 0.0001,
  'n_epochs': 25,
  'eval_freq': 5,
  'scheduler_type': 'MultiStepLR',
  'scheduler_params': {'milestones': [10, 20, 35, 50, 70, 90, 105],
   'gamma': 0.25},
  'loss': {'name': 'CrossEntropyLoss',
   'label_smoothing': False,
   'target_weights': {'y': 1},
   'group_weights': [1],
   'fill_value': -100}},
 'logging': {'mlflow_experiment_name': 'coolgraph',
  'use_mlflow': False,
  'checkpoint_dir': './

In [37]:
# succes
res_runner_with_path = runner_with_path.run()

Sample data: 100%|██████████| 14/14 [00:01<00:00, 12.52it/s]
2024-05-20 15:54:14.951 | INFO     | cool_graph.train.helpers:eval_epoch:199 - test:
 {'accuracy': 0.535, 'cross_entropy': 1.368, 'f1_weighted': 0.465, 'calc_time': 0.005, 'main_metric': 0.535}
2024-05-20 15:54:15.651 | INFO     | cool_graph.train.helpers:eval_epoch:199 - train:
 {'accuracy': 0.532, 'cross_entropy': 1.359, 'f1_weighted': 0.461, 'calc_time': 0.012, 'main_metric': 0.532}
2024-05-20 15:54:22.374 | INFO     | cool_graph.train.helpers:eval_epoch:199 - test:
 {'accuracy': 0.9, 'cross_entropy': 0.315, 'f1_weighted': 0.898, 'calc_time': 0.004, 'main_metric': 0.9}
2024-05-20 15:54:23.135 | INFO     | cool_graph.train.helpers:eval_epoch:199 - train:
 {'accuracy': 0.922, 'cross_entropy': 0.258, 'f1_weighted': 0.92, 'calc_time': 0.013, 'main_metric': 0.922}
2024-05-20 15:54:30.230 | INFO     | cool_graph.train.helpers:eval_epoch:199 - test:
 {'accuracy': 0.916, 'cross_entropy': 0.272, 'f1_weighted': 0.915, 'calc_time': 0

In [38]:
res_runner_with_path["best_loss"]

{'accuracy': 0.921,
 'cross_entropy': 0.29,
 'f1_weighted': 0.921,
 'calc_time': 0.004,
 'main_metric': 0.921,
 'epoch': 20}